from google.colab import drive
drive.mount('/content/drive')

%pip install openai
%pip install ffmpeg-python

In [ ]:
#import json
#%cd /content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing
import pandas as pd
#import numpy as np
import sys
import os
import shutil
#sys.path.insert(0,'/Users/sanjivjha/IMP-OIC/')
from utils.extractframes import FrameExtractor
import graphene
from STAR_eval.gpt_ask import run_gpt
directory_path = 'eval/quant'


In [ ]:
def run_oic():
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    for video in os.scandir(directory_path):
        if video.is_file():
            if not video.name.startswith('.'):
                ex = FrameExtractor(directory_path+'/'+video.name, 1, 5)
                ex.main()
                g = graphene.Graphene(0.3, 0.6)

                if not os.path.isdir(OUT_DIR):
                    os.mkdir(OUT_DIR)
                text = os.path.splitext(video.name)[0]+'graph2text.txt'
                img_path = directory_path+'/'+os.path.splitext(video.name)[0]+'-opencv'


                g.classify_images_window(img_path, 5)
                graph_path = img_path + "/img/JSON"
                g.generate_temporal_graph_frames(graph_path, img_path + "/img")

                g.tg.to_text(os.path.join(OUT_DIR, text))

                if os.path.isfile(os.path.join(OUT_DIR,text)):
                    with open(os.path.join(OUT_DIR,text)) as f:
                        context = "".join(map(str,f.readlines()))
                        return str(context)

In [ ]:
import contextlib
import time
if os.path.isdir(directory_path):
  prompt = run_oic()
  if prompt is not None:
    answer = run_gpt(prompt, 'What the man could be doing doing?')
    print('OIC answer: {}'.format(answer))
  else:
    print('OIC answer: {}'.format(None))

In [ ]:
import shutil

shutil.rmtree('eval/quant/car_out-opencv/')


In [ ]:
import shutil

shutil.rmtree('temp')


In [ ]:

with open(os.path.join('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing/out/Feasibility_T5_981IO712graph2text.txt')) as f:
      context = "".join(map(str,f.readlines()))

      answer = run_gpt(str(context), 'Which object is the person able to hold after closing the window? Options: The book.The food.The mirror.The clothes. ')
      print('OIC answer: {}'.format(answer))
